In [1]:
cd ..

/home/luco/python/ml-imdb


# Movie Rating Prediction from Graph Statistics

In [2]:
import json
import numpy as np

from commonstuff import *

In [3]:
from sklearn.preprocessing import normalize
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error, mean_absolute_error

from keras.models import Sequential
from keras.layers import Input, Dense, Dropout
from keras.activations import relu
from keras.optimizers import SGD

Using TensorFlow backend.


In [4]:
def get_data(datafn, testfn, normalized=True):

    with open(datafn) as file:
        data = json.load(file)
    with open(testfn) as file:
        test = json.load(file)

    x_train = np.array(map(lambda x: np.array(x['attributes'], dtype = 'f'), data['data']))
    x_test = np.array(map(lambda x: np.array(x['attributes'], dtype = 'f'), test['data']))

    if normalized:
        # normalizing by features
        # e.g. betweeness as a very high number corrupts models
        normalize(x_train, axis=0, copy=False)
        normalize(x_test, axis=0, copy=False)
    
    y_train = np.array(map(lambda x: x['target'], data['data']))
    y_train = y_train.reshape(y_train.size,1).ravel()    
    y_test = np.array(map(lambda x: x['target'], test['data']))
    y_test = y_test.reshape(y_test.size,1).ravel()
    
    return x_train, y_train, x_test, y_test

In [13]:
train_fn = "./data_1970_2000.json"
test_fn = "./data_2000_2005.json"

In [14]:
x_train, y_train, x_test, y_test = get_data(train_fn, test_fn)

In [15]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1778, 34), (1778,), (1280, 34), (1280,))

## Support Vector Regression

In [16]:
model = LinearSVR()
model.fit(x_train, y_train)

LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=None, tol=0.0001, verbose=0)

In [17]:
mean_absolute_error(model.predict(x_test), y_test)

1.0456104102921533

## Feedforward Neural Net

In [21]:
model = Sequential()

input_shape = (x_train.shape[1], )

model.add(Dense(160, activation=relu, input_shape=input_shape))
model.add(Dropout(0.4))
model.add(Dense(160, activation=relu))
model.add(Dropout(0.4))
model.add(Dense(1, activation=relu))

In [22]:
model.compile(loss="mse", metrics=["mae"], optimizer="adadelta")

In [23]:
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          epochs=64, batch_size=500, verbose=1)

Train on 1778 samples, validate on 1280 samples
Epoch 1/64
1778/1778 [==============================] - 0s - loss: 39.5150 - mean_absolute_error: 6.1357 - val_loss: 41.0060 - val_mean_absolute_error: 6.2455
Epoch 2/64
1778/1778 [==============================] - 0s - loss: 38.7627 - mean_absolute_error: 6.0738 - val_loss: 39.7234 - val_mean_absolute_error: 6.1420
Epoch 3/64
1778/1778 [==============================] - 0s - loss: 37.2136 - mean_absolute_error: 5.9449 - val_loss: 36.8805 - val_mean_absolute_error: 5.9058
Epoch 4/64
1778/1778 [==============================] - 0s - loss: 33.5522 - mean_absolute_error: 5.6238 - val_loss: 29.8792 - val_mean_absolute_error: 5.2794
Epoch 5/64
1778/1778 [==============================] - 0s - loss: 24.5561 - mean_absolute_error: 4.7407 - val_loss: 14.7952 - val_mean_absolute_error: 3.5865
Epoch 6/64
1778/1778 [==============================] - 0s - loss: 9.4638 - mean_absolute_error: 2.7190 - val_loss: 2.7126 - val_mean_absolute_error: 1.3834


1778/1778 [==============================] - 0s - loss: 2.1179 - mean_absolute_error: 1.1433 - val_loss: 1.7052 - val_mean_absolute_error: 1.0286
Epoch 53/64
1778/1778 [==============================] - 0s - loss: 1.9837 - mean_absolute_error: 1.1043 - val_loss: 1.6998 - val_mean_absolute_error: 1.0250
Epoch 54/64
1778/1778 [==============================] - 0s - loss: 2.0921 - mean_absolute_error: 1.1554 - val_loss: 1.7062 - val_mean_absolute_error: 1.0288
Epoch 55/64
1778/1778 [==============================] - 0s - loss: 2.0934 - mean_absolute_error: 1.1500 - val_loss: 1.7129 - val_mean_absolute_error: 1.0324
Epoch 56/64
1778/1778 [==============================] - 0s - loss: 2.0684 - mean_absolute_error: 1.1446 - val_loss: 1.7359 - val_mean_absolute_error: 1.0455
Epoch 57/64
1778/1778 [==============================] - 0s - loss: 2.0692 - mean_absolute_error: 1.1471 - val_loss: 1.6981 - val_mean_absolute_error: 1.0227
Epoch 58/64
1778/1778 [==============================] - 0s - lo